# Question 2 - IMF Staff Notes
 

https://www.imf.org/en/Publications/SPROLLs/Staff-Discussion-Notes#sort=%40imfdate%20descending

First, import packages and establish paths

In [68]:

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
import pandas as pd 
import time 
import random
import os
import shutil
import glob
import numpy as np
import warnings

warnings.filterwarnings("ignore")
geko_path = "/home/gigilovicu/Documents/masters/semester_2/text_mining/final_project/geckodriver"
download_path = "/home/gigilovicu/Documents/masters/semester_2/text_mining/final_project/"
main_url = "https://www.imf.org/en/publications/search?when=After&series=IMF+Working+Papers&page=251"


Helper functions from class.

In [69]:

#Luis function
def start_up(main_url):
    # check whether I moved to next step; otherwise repeat it:
    # Set aMozilla firefox to automatically download
    profile = webdriver.FirefoxProfile()
    os.makedirs(download_path, exist_ok=True)
    # open browser and set up its preferences:
    browser = ffx_preferences(profile, download_path, geko_path)

    # Enter the website address here
    browser.get(
        main_url)
    time.sleep(seconds())
    return profile, browser, download_path

#Luis function
def ffx_preferences(profile, dfolder, path):
    '''
    Sets the preferences of the firefox browser: download path and the path for the
    selenium executable.
    '''
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    # Set the Mozilla firefox as the browser
    return webdriver.Firefox(firefox_profile=profile, executable_path=path)

#Luis function
def seconds():
    '''
    Generates random waiting time
    '''
    s = 2 +(random.random() * 2)
    return s

#Luis function
def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element_by_xpath(xpath).click()
        elif type == "id":
            browser.find_element_by_id(xpath).click()
        elif type == "css":
            browser.find_element_by_css_selector(xpath).click()
        elif type == "class":
            browser.find_element_by_class_name(xpath).click()
        elif type == "link":
            browser.find_element_by_link_text(xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException):
        return False
    return True

#Luis function
def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(seconds())
        ss += 1
        if ss == 2:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()


`get_url()` retrieves all of the report urls from a particular page by extracting the hyperlink attribute from the title of each report.


In [70]:
#function that gets a list of urls 
def get_url(selenium_web_elements):
    url_list = []
    for i in range(0, len(selenium_web_elements)): 
        if type(selenium_web_elements[i].get_attribute('href')) == str: 
            url_list.append(selenium_web_elements[i].get_attribute('href'))
    return url_list


`page_scroll()` retrieves the urls for a specified number of pages on the website. It finds the web element where the url is embedded, extracts it, appends it to a list and then moves to the next page. Sometimes, a a 'give feedback' prompt pops up, which the code will check for and close if necessary. 

`download_and_get_metadata()` iterates through opening each url extracted using the previous function and downloads the metadata on that page. This includes the title of the report, authors, publication date, summary (like an abstract), page length and subject tags. It also downloads a PDF of the report. The function returns a dataframe containing the metadata associated with each url.

One article contained a different (historical) page layout, so we implement a try/except to pass over it. The try/except also handles pages that contain the correct html format, but for whatever reason do not have all of the metadata (for example, not every article has subject tags). In the try/except it is important to first download the file before extracting the metadata. If download is not called first and some metadata is missing, the code will pass over downloading the file.


In [71]:
#function that gets all metadata and downloads file 
def download_and_get_metadata(report_df):
    for url in range(0, len(report_df['url'])): 
        time.sleep(seconds())  
        # Check if it contains
        if ('https://www.imf.org' in report_df.at[url, 'url']):
            open_url = report_df.at[url, 'url']
            print(open_url)
            browser.get(open_url)
        else:
            open_url = 'https://www.imf.org' + report_df.at[url, 'url']
            print(open_url)
            browser.get(open_url)
        check_and_click(browser, '//*[@class="fsrButton fsrButton__inviteDecline fsrDeclineButton"]', type = 'xpath') 
        #check if feedback box pops up. If so click close. 
        #wrapped in a try/except because there are some subtelties. See below for details. 
        try:
            report_df.at[url, 'title'] = browser.find_elements_by_xpath('//*[@class="conf"]')[0].text
        except: 
            report_df.at[url, 'title'] = np.nan
        try:
            report_df.at[url, 'authors'] = browser.find_elements_by_xpath('//*[@class="pub-desc hide"]')[0].text
        except:
            report_df.at[url, 'authors'] = np.nan
        try: 
            report_df.at[url, 'publication_date'] = browser.find_elements_by_xpath('//*[@class="pub-desc hide"]')[1].text
        except: 
            report_df.at[url, 'publication_date'] =np.nan
        try:
            resume=''
            for i in browser.find_elements_by_xpath('//*[@class="pub-desc"]'): 
                if len(i.text)>=len(resume): 
                    resume=i.text
            report_df.at[url, 'summary'] = resume
        except:
            report_df.at[url, 'summary'] = np.nan
        try:
            report_df.at[url, 'length'] = browser.find_elements_by_xpath('//*[@class="pub-desc"]')[-1].text
        except: 
            report_df.at[url, 'length'] =np.nan
        try:
            report_df.at[url, 'subjects'] = browser.find_elements_by_xpath('//*[@class="pub-desc publinks"]')[0].text
        except:
            report_df.at[url, 'subjects'] =np.nan
    return report_df

In [72]:
#function that scrolls through pages and retrieves all the url's 
def page_scroll(starting_point,end_point):
    browser.get(main_url)
    
    #go to the initial page
    for i in range(1,starting_point): 
        #check if feedback box pops up. If so click close. 
        check_and_click(browser, '//*[@class="fsrButton fsrButton__inviteDecline fsrDeclineButton"]', type = 'xpath') 
        check_and_click(browser, '//*[@class="next"]', type = 'xpath')

    for i in range(starting_point, end_point+1):
        reports = []
        report_urls = []
        for j in range(10):
            reports += browser.find_elements_by_css_selector(f'div.result-row:nth-child({j+5}) > h6:nth-child(1) > a:nth-child(2)')
        time.sleep(seconds())
        report_urls += [i.get_attribute('href') for i in reports]        
        #get the metadata
        report_df = pd.DataFrame(columns = ['title', 'authors', 'publication_date', 'summary', 'length', 'subjects', 'url'])
        report_df['url'] = report_urls
        #save the last page
        last_url=browser.current_url
        new_df=download_and_get_metadata(report_df)
        try: 
            data_final=pd.read_csv(download_path + 'urls_final'+'.csv',sep='|')
            data_final=data_final.append(new_df)
            data_final.to_csv(download_path + 'urls_final'+'.csv',index=False,sep='|',quotechar='"')
        except: 
            new_df.to_csv(download_path + 'urls_final'+'.csv',index=False,sep='|',quotechar='"')
        # come back to the last url 
        browser.get(last_url)
        print(i)
        time.sleep(seconds())
        #check if feedback box pops up. If so click close. 
        check_and_click(browser, '//*[@class="fsrButton fsrButton__inviteDecline fsrDeclineButton"]', type = 'xpath') 
        check_and_click(browser, '//*[@class="next"]', type = 'xpath')

#simple function to flatten the list of urls 
def flatten(t):
    return [item for sublist in t for item in sublist]


In [73]:
#profile, browser, download_path = start_up(main_url)
#browser.get("https://www.imf.org/en/publications/search?when=After&series=IMF+Working+Papers&page=251")
#reports = []
#report_urls = []
#for j in range(10):
#    reports += browser.find_elements_by_css_selector(f'div.result-row:nth-child({j+5}) > h6:nth-child(1) > a:nth-child(2)')
#report_urls += [i.get_attribute('href') for i in reports]



All the files that are downloaded are named randomly. `rename_file()` extracts the names of the downloaded files into a list and sorts it based on the 'last modified' date/time (earliest first). It only does this for pdf files in the directory. The list is then input into a dataframe with the other metadata, which is also sorted in the order it was extracted. The function then renames the files with the article's title.

Finally, using all the functions created the code below scrapes the metadata and pdfs from the IMF website.

The final output is a csv with all the metadata and a directory with all the PDFs downloaded. The next step in this process would be to read in the text from the PDFs. Overall we extracted 116 papers from 12 pages of staff notes.

In [74]:
#Change for staring Page 
starting_page=1
#Change for final Page
final_page=250
#-------------------------------------------------------------------------------------------------------------------------------------
profile, browser, download_path = start_up(main_url)
page_scroll(starting_page,final_page)


https://www.imf.org/en/Publications/WP/Issues/2016/12/31/Information-Rigidities-in-Economic-Growth-Forecasts-Evidence-from-a-Large-International-Panel-40358
https://www.imf.org/en/Publications/WP/Issues/2016/12/31/Getting-to-Know-GIMF-The-Simulation-Properties-of-the-Global-Integrated-Monetary-and-Fiscal-40357
https://www.imf.org/en/Publications/WP/Issues/2016/12/31/Systemic-Contingent-Claims-Analysis-Estimating-Market-Implied-Systemic-Risk-40356
https://www.imf.org/en/Publications/WP/Issues/2016/12/31/The-Quality-of-the-Recent-High-Growth-Episode-in-Sub-Saharan-Africa-40348
https://www.imf.org/en/Publications/WP/Issues/2016/12/31/Outward-Foreign-Direct-Investment-and-Domestic-Investment-The-Case-of-Developing-Countries-40347
https://www.imf.org/en/Publications/WP/Issues/2016/12/31/Benchmarking-Banking-Sector-Efficiency-Across-Regional-Blocks-in-Sub-Saharan-Africa-What-40344
https://www.imf.org/en/Publications/WP/Issues/2016/12/31/The-Endogenous-Skill-Bias-of-Technical-Change-and-Inequ

In [8]:
# validation 
data_val=pd.read_csv(download_path + 'urls_final'+'.csv',sep='|')
print(data_val.shape)
data_val.head()

(10, 7)


,title,authors,publication_date,summary,length,subjects,url
0,Information Rigidities in Economic Growth Fore...,Jonas Dovern ; Ulrich Fritsche ; Prakash Loung...,"February 27, 2013",We examine the behavior of forecasts for real ...,24,Developed countries Economic forecasting Econo...,https://www.imf.org/en/Publications/WP/Issues/...
1,Getting to Know GIMF : The Simulation Properti...,Derek Anderson ; Benjamin L Hunt ; Mika Kortel...,"February 27, 2013",The Global Integrated Monetary and Fiscal mode...,66,Demand Economic models External shocks Financi...,https://www.imf.org/en/Publications/WP/Issues/...
2,Systemic Contingent Claims Analysis : Estimati...,Andreas A. Jobst ; Dale F. Gray,"February 27, 2013",The recent global financial crisis has forced ...,93,Banks Contingent liabilities Financial crises ...,https://www.imf.org/en/Publications/WP/Issues/...
3,The Quality of the Recent High-Growth Episode ...,Marcelo Martinez ; Montfort Mlachila,"February 26, 2013",The paper explores the quality of the recent h...,36,Africa Economic growth Education Health care P...,https://www.imf.org/en/Publications/WP/Issues/...
4,Outward Foreign Direct Investment and Domestic...,Ali J Al-Sadiq,"February 26, 2013","Over the past two decades, the growth rate of ...",28,Capital flows Developing countries Economic gr...,https://www.imf.org/en/Publications/WP/Issues/...
